# Part 1

In [8]:
boss_hp = 71
boss_dmg = 10

player_hp = 50
player_mana = 500

missile_mana = 53
missile_dmg = 4

drain_mana = 73
drain_dmg = 2

shield_mana = 113
shield_duration = 6
shield_armor = 7

poison_mana = 173
poison_dmg = 3
poison_duration = 6

recharge_mana = 229
recharge_duration = 5
recharge_refill = 101


class GameSimulation(object):
    
    def __init__(self, player_hp, player_mana, boss_hp, total_cost, past_spells, active_effects, effects_durations):
        self.boss_hp = boss_hp
        self.boss_dmg = boss_dmg
        self.player_hp = player_hp
        self.player_mana = player_mana
        self.player_armor = 0
        self.total_cost = total_cost
        #self.won_simulations = []
        #self.lost_simulations = []
        self.spell_order = past_spells
        self.active_effects = active_effects
        self.effects_durations = effects_durations
        self.spell_costs = [missile_mana, drain_mana, shield_mana, poison_mana, recharge_mana]
        self.spells = ['Missile', 'Drain', 'Shield', 'Poison', 'Recharge']
        
        
    def cast_missile(self):
        self.boss_hp -= missile_dmg
        self.spell_order.append('Missile')
        self.player_mana -= missile_mana
        self.total_cost += missile_mana
        if self.boss_hp <= 0:
            #self.won_simulations.append(self.total_cost)
            return 'Win'
        
    
    def cast_drain(self):
        self.boss_hp -= drain_dmg
        self.spell_order.append('Drain')
        self.player_hp += drain_dmg
        self.player_mana -= drain_mana
        self.total_cost += drain_mana
        if self.boss_hp <= 0:
            #self.won_simulations.append(self.total_cost)
            return 'Win'
        
        
    def cast_shield(self):
        self.active_effects.append('Shield')
        self.effects_durations.append(shield_duration)
        self.spell_order.append('Shield')
        self.player_mana -= shield_mana
        self.total_cost += shield_mana
        
    
    def cast_poison(self):
        self.active_effects.append('Poison')
        self.spell_order.append('Poison')
        self.effects_durations.append(poison_duration)
        self.player_mana -= poison_mana
        self.total_cost += poison_mana
        
    
    def cast_recharge(self):
        self.active_effects.append('Recharge')
        self.spell_order.append('Recharge')
        self.effects_durations.append(recharge_duration)
        self.player_mana -= recharge_mana
        self.total_cost += recharge_mana
        
    
    def round_start(self):
        self.player_armor = 0
        for i, eff in enumerate(self.active_effects):
            self.effects_durations[i] -= 1
            if eff == 'Shield':
                self.player_armor = shield_armor
            elif eff == 'Poison':
                self.boss_hp -= poison_dmg
                if self.boss_hp <= 0:
                    #self.won_simulations.append(self.total_cost)
                    return 'Win'
            else: # Recharge
                self.player_mana += recharge_refill
        self.active_effects = [e for i, e in enumerate(self.active_effects) if self.effects_durations[i] != 0]
        self.effects_durations = [d for d in self.effects_durations if d != 0]
        available_spells = [spell for i, spell in enumerate(self.spells) if self.player_mana >= self.spell_costs[i] and spell not in self.active_effects]
        return available_spells if available_spells != [] else 'Loss'
        
        
    def boss_attack(self):
        self.player_hp -= (self.boss_dmg - self.player_armor)
        if self.player_hp <= 0:
            #self.lost_simulations.append(self.total_cost)
            return 'Loss'        
    
    

class Experimenter(object):
    
    def __init__(self):
        self.won_simulations = []
        self.lost_simulations = []
        self.simulations = []
        
    
    def run_simulations(self, n_rounds):
        GS = GameSimulation(player_hp, player_mana, boss_hp, 0, [], [], [])
        self.simulations = [GS]
        for k in range(n_rounds):
            print(f"TURN {k+1}: {len(self.simulations)} scenarios - {len(self.won_simulations)} won, {len(self.lost_simulations)} lost")
            new_sims = [] 
            for game in self.simulations:
                
                # Start of player turn
                spells = game.round_start()
                if spells == 'Win':
                    self.won_simulations.append(game)
                elif spells == 'Loss':
                    self.lost_simulations.append(game)
                else:
                    # Start of player attack
                    for spell in spells:
                        new_game = GameSimulation(game.player_hp, 
                                                  game.player_mana, 
                                                  game.boss_hp, 
                                                  game.total_cost, 
                                                  game.spell_order.copy(),
                                                  game.active_effects.copy(),
                                                  game.effects_durations.copy())
                        # Casting a spell
                        if spell == 'Missile':
                            attack = new_game.cast_missile()
                        elif spell == 'Drain':
                            attack = new_game.cast_drain()
                        elif spell == 'Shield':
                            attack = new_game.cast_shield()
                        elif spell == 'Poison':
                            attack = new_game.cast_poison()
                        else: #'Recharge'
                            attack = new_game.cast_recharge()
                        # Result of player attack    
                        if attack == 'Win':
                            self.won_simulations.append(new_game)
                        else:
                            # Start of boss turn
                            boss_round = new_game.round_start()
                            if boss_round == 'Win':
                                self.won_simulations.append(new_game)
                            else:
                                # Start of boss attack
                                boss_attack = new_game.boss_attack()
                                if boss_attack == 'Loss':
                                    self.lost_simulations.append(new_game)
                                else:
                                    # End of round
                                    new_sims.append(new_game)
                                    
            # Replacing the simulations
            self.simulations = new_sims
    
                                

console = Experimenter()
console.run_simulations(13)


costs = [won_sim.total_cost for won_sim in console.won_simulations]
print(f"FINAL ANSWER: {min(costs)}")

TURN 1: 1 scenarios - 0 won, 0 lost
TURN 2: 5 scenarios - 0 won, 0 lost
TURN 3: 22 scenarios - 0 won, 0 lost
TURN 4: 84 scenarios - 0 won, 0 lost
TURN 5: 312 scenarios - 0 won, 0 lost
TURN 6: 1003 scenarios - 0 won, 52 lost
TURN 7: 2071 scenarios - 0 won, 1244 lost
TURN 8: 6719 scenarios - 0 won, 1839 lost
TURN 9: 10655 scenarios - 0 won, 15913 lost
TURN 10: 28388 scenarios - 0 won, 20689 lost
TURN 11: 30149 scenarios - 0 won, 79000 lost
TURN 12: 64514 scenarios - 0 won, 111883 lost
TURN 13: 105672 scenarios - 11 won, 229672 lost
FINAL ANSWER: 1824


# Part 2

In [9]:
boss_hp = 71
boss_dmg = 10

player_hp = 50
player_mana = 500

missile_mana = 53
missile_dmg = 4

drain_mana = 73
drain_dmg = 2

shield_mana = 113
shield_duration = 6
shield_armor = 7

poison_mana = 173
poison_dmg = 3
poison_duration = 6

recharge_mana = 229
recharge_duration = 5
recharge_refill = 101


class GameSimulation(object):
    
    def __init__(self, player_hp, player_mana, boss_hp, total_cost, past_spells, active_effects, effects_durations):
        self.boss_hp = boss_hp
        self.boss_dmg = boss_dmg
        self.player_hp = player_hp
        self.player_mana = player_mana
        self.player_armor = 0
        self.total_cost = total_cost
        #self.won_simulations = []
        #self.lost_simulations = []
        self.spell_order = past_spells
        self.active_effects = active_effects
        self.effects_durations = effects_durations
        self.spell_costs = [missile_mana, drain_mana, shield_mana, poison_mana, recharge_mana]
        self.spells = ['Missile', 'Drain', 'Shield', 'Poison', 'Recharge']
        
        
    def cast_missile(self):
        self.boss_hp -= missile_dmg
        self.spell_order.append('Missile')
        self.player_mana -= missile_mana
        self.total_cost += missile_mana
        if self.boss_hp <= 0:
            #self.won_simulations.append(self.total_cost)
            return 'Win'
        
    
    def cast_drain(self):
        self.boss_hp -= drain_dmg
        self.spell_order.append('Drain')
        self.player_hp += drain_dmg
        self.player_mana -= drain_mana
        self.total_cost += drain_mana
        if self.boss_hp <= 0:
            #self.won_simulations.append(self.total_cost)
            return 'Win'
        
        
    def cast_shield(self):
        self.active_effects.append('Shield')
        self.effects_durations.append(shield_duration)
        self.spell_order.append('Shield')
        self.player_mana -= shield_mana
        self.total_cost += shield_mana
        
    
    def cast_poison(self):
        self.active_effects.append('Poison')
        self.spell_order.append('Poison')
        self.effects_durations.append(poison_duration)
        self.player_mana -= poison_mana
        self.total_cost += poison_mana
        
    
    def cast_recharge(self):
        self.active_effects.append('Recharge')
        self.spell_order.append('Recharge')
        self.effects_durations.append(recharge_duration)
        self.player_mana -= recharge_mana
        self.total_cost += recharge_mana
        
    
    def round_start(self):
        self.player_armor = 0
        for i, eff in enumerate(self.active_effects):
            self.effects_durations[i] -= 1
            if eff == 'Shield':
                self.player_armor = shield_armor
            elif eff == 'Poison':
                self.boss_hp -= poison_dmg
                if self.boss_hp <= 0:
                    #self.won_simulations.append(self.total_cost)
                    return 'Win'
            else: # Recharge
                self.player_mana += recharge_refill
        self.active_effects = [e for i, e in enumerate(self.active_effects) if self.effects_durations[i] != 0]
        self.effects_durations = [d for d in self.effects_durations if d != 0]
        available_spells = [spell for i, spell in enumerate(self.spells) if self.player_mana >= self.spell_costs[i] and spell not in self.active_effects]
        return available_spells if available_spells != [] else 'Loss'
        
        
    def boss_attack(self):
        self.player_hp -= (self.boss_dmg - self.player_armor)
        if self.player_hp <= 0:
            #self.lost_simulations.append(self.total_cost)
            return 'Loss'        
    
    

class Experimenter(object):
    
    def __init__(self):
        self.won_simulations = []
        self.lost_simulations = []
        self.simulations = []
        
    
    def run_simulations(self, n_rounds):
        GS = GameSimulation(player_hp, player_mana, boss_hp, 0, [], [], [])
        self.simulations = [GS]
        for k in range(n_rounds):
            print(f"TURN {k+1}: {len(self.simulations)} scenarios - {len(self.won_simulations)} won, {len(self.lost_simulations)} lost")
            new_sims = [] 
            for game in self.simulations:
                
                # Start of player turn
                game.player_hp -= 1
                if game.player_hp <= 0:
                    self.lost_simulations.append(game)
                else:
                    spells = game.round_start()
                    if spells == 'Win':
                        self.won_simulations.append(game)
                    elif spells == 'Loss':
                        self.lost_simulations.append(game)
                    else:
                        # Start of player attack
                        for spell in spells:
                            new_game = GameSimulation(game.player_hp, 
                                                      game.player_mana, 
                                                      game.boss_hp, 
                                                      game.total_cost, 
                                                      game.spell_order.copy(),
                                                      game.active_effects.copy(),
                                                      game.effects_durations.copy())
                            # Casting a spell
                            if spell == 'Missile':
                                attack = new_game.cast_missile()
                            elif spell == 'Drain':
                                attack = new_game.cast_drain()
                            elif spell == 'Shield':
                                attack = new_game.cast_shield()
                            elif spell == 'Poison':
                                attack = new_game.cast_poison()
                            else: #'Recharge'
                                attack = new_game.cast_recharge()
                            # Result of player attack    
                            if attack == 'Win':
                                self.won_simulations.append(new_game)
                            else:
                                # Start of boss turn
                                boss_round = new_game.round_start()
                                if boss_round == 'Win':
                                    self.won_simulations.append(new_game)
                                else:
                                    # Start of boss attack
                                    boss_attack = new_game.boss_attack()
                                    if boss_attack == 'Loss':
                                        self.lost_simulations.append(new_game)
                                    else:
                                        # End of round
                                        new_sims.append(new_game)
                                    
            # Replacing the simulations
            self.simulations = new_sims
    
                                

console = Experimenter()
console.run_simulations(15)


costs = [won_sim.total_cost for won_sim in console.won_simulations]
print(f"FINAL ANSWER: {min(costs)}")

TURN 1: 1 scenarios - 0 won, 0 lost
TURN 2: 5 scenarios - 0 won, 0 lost
TURN 3: 22 scenarios - 0 won, 0 lost
TURN 4: 84 scenarios - 0 won, 0 lost
TURN 5: 312 scenarios - 0 won, 0 lost
TURN 6: 745 scenarios - 0 won, 310 lost
TURN 7: 1749 scenarios - 0 won, 700 lost
TURN 8: 2632 scenarios - 0 won, 4346 lost
TURN 9: 6741 scenarios - 0 won, 6169 lost
TURN 10: 7566 scenarios - 0 won, 18388 lost
TURN 11: 8434 scenarios - 0 won, 27482 lost
TURN 12: 6101 scenarios - 0 won, 50345 lost
TURN 13: 7395 scenarios - 0 won, 59173 lost
TURN 14: 3770 scenarios - 1 won, 72491 lost
TURN 15: 919 scenarios - 1 won, 78257 lost
FINAL ANSWER: 1937
